In [18]:
import time
from functools import cache
from array import array

unsafe_composite = 5261933844650100908430030083398098838688018147149529533465444719385566864605781576487305356717074882505882701585297765789323726258356035692769897420620858774763694117634408028918270394852404169072671551096321238430993811080749636806153881798472848720411673994908247486124703888115308603904735959457057925225503197625820670522050494196703154086316062123787934777520599894745147260327060174336101658295022275013051816321617046927321006322752178354002696596328204277122466231388232487691224076847557856202947748540263791767128195927179588238799470987669558119422552470505956858217654904628177286026365989987106877656917

# Definiere Dekorierer Funktion um die Laufzeit von den Algorithmen ausgeben zu können.
def time_function(func):
    def wrapper(**kwargs):
        start = time.time()
        result = func(**kwargs)
        delta = time.time() - start
        print(f'Funktion {func.__name__} hat {delta} Sekunden benötigt!')
        return result
    return wrapper

In [19]:
@cache
def get_primes(n: int) -> list:
    # Für die Differenzen reichen shorts aus.
    p_dist = array("H")
    last_prime = 2
    # Sieb Implementation in der Vielfache eliminiert werden.
    # Es wird zur Speicherkostenbegrenzung nur der Abstand zur letzten Primzahl gespeichert.
    primes = [True] * (n-2)
    for number in range(2, n):
        if primes[number-2]:
            p_dist.append(int((number-last_prime)))
            last_prime = number
            for multiple in range(2 * number,n, number):
                primes[multiple - 2] = False
    return p_dist

# Lookup Table für nachfolgende Beispiele
primes_lut = get_primes(n=1000000000)

prime_pair = 999999929, 999999937
composite = prime_pair[0]*prime_pair[1]

In [20]:
@time_function
def probe_division(number: int, interval: tuple):
    assert number > 1
    n = number
    prime_factor = 2
    factors = []
    for difference in primes_lut:
        # 1 ist trivialer Faktor
        if n == 1:
            break
        prime_factor = prime_factor + difference
        if prime_factor < interval[0]:
            continue
        # Produkt zweier Faktoren größer sqrt(n) wären größer als n.
        # Daher kann in dem Fall abgebrochen werden
        if prime_factor**2 > n or prime_factor >= interval[1]:
            factors.append(int(n))
            break
        # Faktor so lange rausdividieren, bis er n nicht mehr teilt.
        while n % prime_factor == 0:
            n = int(n/prime_factor)
            factors.append(prime_factor)
    return factors

# Beispiel aus dem Vortrag: 999999866000004473
print(f'Zusammengesetzte Zahl: {composite}')
probe_division(number=composite, interval=(2,2000000000))

Zusammengesetzte Zahl: 999999866000004473
Funktion probe_division hat 12.572450399398804 Sekunden benötigt!


[999999929, 999999937]

In [21]:
@cache
def ggT(number1: int, number2: int):
    h = number1 % number2
    a,b = number2, h

    while b != 0:
        h = a % b
        a = b
        b = h
    return abs(a)

# Test mit 2 Primzahlen
assert ggT(99999989, 99999971) == 1
ggT(65536, 48)

16

In [22]:
from math import floor, ceil, sqrt

root_n = lambda x, n: x**(1./n) if 0 <= x else -(-x)**(1./n)

@time_function
def lehmann_factor(n: int):
    factors = probe_division(number=n, interval=(2, int(floor(root_n(n,3)))))
    if len(factors) > 1:
        print(f'Faktoren kleiner als {root_n(n,3)}')
        return factors
    for k in range(1, int(ceil(root_n(n,3))) + 1):
        for x in range( ceil(sqrt(4*k*n)), floor(sqrt(4*k*n) + root_n(n,6)/4*sqrt(k)) ):
            y = x**2 - 4*k*n
            if sqrt(y) % 1 == 0:
                return ggT(x+int(sqrt(y)),n)


factor = lehmann_factor(n=composite)
assert factor in prime_pair
print(f'{factor} ist ein Faktor von {composite}')

lehmann_factor(n=17*23)

Funktion probe_division hat 0.018770456314086914 Sekunden benötigt!
Funktion lehmann_factor hat 0.023241043090820312 Sekunden benötigt!
999999937 ist ein Faktor von 999999866000004473
Funktion probe_division hat 6.4373016357421875e-06 Sekunden benötigt!
Funktion lehmann_factor hat 0.00010061264038085938 Sekunden benötigt!


17

In [27]:
from math import isqrt
from gmpy2 import is_square, is_prime, invert, gcd

@time_function
def fermat_factor(n: int):
    print(f'{n.bit_length()} bits')
    a = isqrt(n) + 1
    b2 = 3
    while not is_square(b2):
        b2 = a**2 - n
        a = a + 1
    # 1 von a abziehen, damit wir die Abbruchbedingung im Schleifenkopf haben können.
    a = a - 1
    b = isqrt(b2)
    p, q = (a+b), (a-b)
    return p, q

p, q = fermat_factor(n=composite)
assert is_prime(p) and is_prime(q)
print(f'{p}\n{q}')

60 bits
Funktion fermat_factor hat 7.200241088867188e-05 Sekunden benötigt!
999999937
999999929


Um zu zeigen, dass die Fermat Faktorisierung für nah beieinander gewählte Zahlen besonders schnell funktioniert, prüfen wir eine 2046 bit lange Zahl.
Die Faktoren dieser Zahl sind in der ersten Hälfte ihrer Bits gleich.

In [24]:
p, q = fermat_factor(n=unsafe_composite)
assert is_prime(p) and is_prime(q)
assert unsafe_composite == p * q
print(f'p: {p}\nq: {q}')

2046 bits
Funktion fermat_factor hat 0.00011587142944335938 Sekunden benötigt!
p: 72539188337409048434517657668785982436503618029818802387833126880251213106684983301847459281756173872849655980341983435213476251581941251979385718844779855101287148374206957436458915587712518501281793789555480805845328694482152421962093714097210685267495028743960484986044572019270471629952251128834754752071
q: 72539188337409048434517657668785982436503618029818802387833126880251213106684983301847459281756173872849655980341983435213476251581941251979385718844779768486519862521371761417707655650528352916168732086751886502287478577426433344249124093776641317837723657300923622528678618140782421245730805689484709681027


Wir können nun relativ schnell mithilfe der Multiplikativen Inversen modulo $\Phi(N)$ den PrivateKey berechnen.

In [25]:
import rsa

e = 65537
N = unsafe_composite

# Geheime Nachricht!
message = "Hallo Bob!".encode('utf8')
encrypted = rsa.encrypt(message=message, pub_key=rsa.PublicKey(n=unsafe_composite, e=e))

p, q = fermat_factor(n=unsafe_composite)
phi_n = (p-1)*(q-1)
d = invert(e,phi_n)

print(f'PrivateKey: {d}')

# Hier bauen wir unseren PrivateKey nur aus den Infos des PublicKeys zusammen.  
priv_key = rsa.PrivateKey(unsafe_composite, e, d, p, q)

# Nun schauen wir, ob unsere Entschlüsselung funktioniert hat.
decrypted = rsa.decrypt(crypto=encrypted, priv_key=priv_key)
assert message == decrypted
print(decrypted.decode('utf8'))


2046 bits
Funktion fermat_factor hat 0.00010895729064941406 Sekunden benötigt!
PrivateKey: 1780018513753951769838316782106838141106754387938188653080380692262050710107422945064979473555663978289445954104491378420576270063441312713561936399517287013999894694416204983461526384391684093387569285866082394006670015283888787219317813746008255735409414719427435597714034594191317755597113023500663353559763027049659940922933197177165188452818904235179156023616509434680097898474152187396851510136984834330488197837586193827324124894033485191502110575202080534184710849113052688635695330872004479190296678616047636148373098654002443273529175473336753354574595877161023282772845813963994656394870693323649494959673
Hallo Bob!


In [37]:
g = lambda x,c,n: (x**2 - c) % n

@time_function
def pollard_rho(n: int):
    t, h, c = 2, 2, 1
    d = 0
    while True:
        
        c = 1
        while True:
            t = g(t,c,n)
            h = g(g(h,c,n),c,n)
            d = gcd(t-h, n)

            if d == n:
                d = False
            if d > 1:
                break
        
        if d:
            return d
        c += 1

from rsa.parallel import getprime

pollard_rho(n=composite)

Funktion pollard_rho hat 0.05369400978088379 Sekunden benötigt!


mpz(999999929)

Wir testen die Behauptung, dass der Pollard Rho Algorithmus für Primzahl Paare, bei denen der erste Faktor kleiner ist.
Dazu multiplizieren wir eine 256 bit Primzahl mit unserer 30 bit Primzahl

In [43]:
large_composite = prime_pair[0] * getprime(256, 16)
pollard_rho(n=large_composite)

Funktion pollard_rho hat 0.12988519668579102 Sekunden benötigt!


mpz(999999929)

Ein weiteres schönes Beispiel dafür ist die 8. Fermat Zahl, welche einen recht kleinen ersten Faktor hat.

In [45]:
fermat_8 = 115792089237316195423570985008687907853269984665640564039457584007913129639937
pollard_rho(n=fermat_8)

Funktion pollard_rho hat 20.265902042388916 Sekunden benötigt!


mpz(1238926361552897)